# Automated CapsnetsS2I train + test

First, import stuff

In [9]:
import model_s2i
import data_loader
import tensorflow as tf

In [10]:
import warnings
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

## Load word vectors and data

In [11]:
WORD_VEC_SCENARIO = "FastText-300-verbsDatasets-synAntVSP-dia"
SCENARIO_NAME = '1-attention-head'

word2vec_path = '../../romanian_word_vecs/cleaned-vectors-diacritice.vec'

# No diacritics data
training_data_paths = [
#         '../data-capsnets/scenario0/train.txt',
#         '../data-capsnets/scenario1/train.txt',
#         '../data-capsnets/scenario2/train.txt',
#         '../data-capsnets/scenario3.1/train.txt',
#         '../data-capsnets/scenario3.2/train.txt',
#         '../data-capsnets/scenario3.3/train.txt',
        '../data-capsnets/diacritics/scenario0/train.txt',
        '../data-capsnets/diacritics/scenario1/train.txt',
        '../data-capsnets/diacritics/scenario2/train.txt',
        '../data-capsnets/diacritics/scenario31/train.txt',
        '../data-capsnets/diacritics/scenario32/train.txt',
        '../data-capsnets/diacritics/scenario33/train.txt',
        
    ]
test_data_paths = [
#     '../data-capsnets/scenario0/test.txt',
#     '../data-capsnets/scenario1/test.txt',
#     '../data-capsnets/scenario2/test.txt',
#     '../data-capsnets/scenario3.1/test.txt',
#     '../data-capsnets/scenario3.2/test.txt',
#     '../data-capsnets/scenario3.3/test.txt',
    '../data-capsnets/diacritics/scenario0/test.txt',
    '../data-capsnets/diacritics/scenario1/test.txt',
    '../data-capsnets/diacritics/scenario2/test.txt',
    '../data-capsnets/diacritics/scenario31/test.txt',
    '../data-capsnets/diacritics/scenario32/test.txt',
    '../data-capsnets/diacritics/scenario33/test.txt',
]

scenario_nums = [
#     '0_' + WORD_VEC_SCENARIO,
#     '1_' + WORD_VEC_SCENARIO,
#     '2_' + WORD_VEC_SCENARIO,
#     '31_' + WORD_VEC_SCENARIO,
#     '32_' + WORD_VEC_SCENARIO,
#     '33_' + WORD_VEC_SCENARIO,
#     '0_dia_' + WORD_VEC_SCENARIO,
#     '1_dia_' + WORD_VEC_SCENARIO,
#     '2_dia_' + WORD_VEC_SCENARIO,
#     '31_dia_' + WORD_VEC_SCENARIO,
#     '32_dia_' + WORD_VEC_SCENARIO,
#     '33_dia_' + WORD_VEC_SCENARIO,
    '0-' + SCENARIO_NAME,
    '1-' + SCENARIO_NAME,
    '2-' + SCENARIO_NAME,
    '31-' + SCENARIO_NAME,
    '32-' + SCENARIO_NAME,
    '33-' + SCENARIO_NAME,
]

In [12]:
print('------------------load word2vec begin-------------------')
w2v = data_loader.load_w2v(word2vec_path)
print('------------------load word2vec end---------------------')

------------------load word2vec begin-------------------
loading time took 000.06
------------------load word2vec end---------------------


# Flag setting functions + utils

In [13]:
import csv

def set_flags(data, scenario):
    FLAGS = tf.app.flags.FLAGS

    tf.app.flags.DEFINE_boolean('save_model', False, 'save model to disk')
    tf.app.flags.DEFINE_string('summaries_dir', './logs', 'tensorboard summaries')
    tf.app.flags.DEFINE_string('ckpt_dir', './saved_models/', 'check point dir')
    tf.app.flags.DEFINE_string('scenario_num', scenario, 'Scenario number')
    tf.app.flags.DEFINE_string('errors_dir', './errors/', 'Errors dir')
    tf.app.flags.DEFINE_string('results_dir', './results/', 'Results dir')
    tf.app.flags.DEFINE_string('hyperparams_dir', './hyperparams/', 'Hyperparams dir')
    
    vocab_size, word_emb_size = data['embedding'].shape
    _, max_sentence_length = data['x_tr'].shape
    intents_number = len(data['intents_dict'])
    slots_number = len(data['slots_dict'])
    hidden_size = 64

    tf.app.flags.DEFINE_float('keep_prob', 1, 'embedding dropout keep rate for training')
    tf.app.flags.DEFINE_integer('hidden_size', hidden_size, 'embedding vector size')
    tf.app.flags.DEFINE_integer('batch_size', 32, 'batch size')
    tf.app.flags.DEFINE_integer('num_epochs', 20, 'num of epochs')
    tf.app.flags.DEFINE_integer('vocab_size', vocab_size, 'vocab size of word vectors')
    tf.app.flags.DEFINE_integer('max_sentence_length', max_sentence_length, 'max number of words in one sentence')
    tf.app.flags.DEFINE_integer('intents_nr', intents_number, 'intents_number')  #
    tf.app.flags.DEFINE_integer('slots_nr', slots_number, 'slots_number')  #
    tf.app.flags.DEFINE_integer('word_emb_size', word_emb_size, 'embedding size of word vectors')
    tf.app.flags.DEFINE_boolean('use_embedding', True, 'whether to use embedding or not.')
    tf.app.flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
    tf.app.flags.DEFINE_integer('slot_routing_num', 2, 'slot routing num')
    tf.app.flags.DEFINE_integer('intent_routing_num', 3, 'intent routing num')
    tf.app.flags.DEFINE_integer('intent_output_dim', 16, 'intent output dimension')
    tf.app.flags.DEFINE_integer('slot_output_dim', 2 * hidden_size, 'slot output dimension')
    tf.app.flags.DEFINE_integer('d_a', 20, 'self attention weight hidden units number')
    tf.app.flags.DEFINE_integer('r', 3, 'number of self attention heads')
    tf.app.flags.DEFINE_float('alpha', 0.0001, 'coefficient for self attention loss')
    tf.app.flags.DEFINE_integer('n_splits', 3, 'Number of cross-validation splits')
    tf.app.flags.DEFINE_float('rerouting_coef', 0.5, 'coefficient for rerouting')
    tf.app.flags.DEFINE_boolean('use_rerouting', True, 'whether to use rerouting or not')
    tf.app.flags.DEFINE_boolean('use_attention', True,
                                'whether to use attention or not. If attention is used,'
                                'slot_output_dim will be overridden to hidden_size * 2')
    
    tf.app.flags.DEFINE_string('f', '', 'kernel') # Without this there's an error: unknown command line flag 'f'
    
    return FLAGS

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
def write_csv(run_results):
    with open('results/results_{}.csv'.format(SCENARIO_NAME), 'w') as f:
        header_line = ['Scenario', 'Intent F1', 'Slot F1']
        writer = csv.writer(f)
        writer.writerow(header_line)
        for k, v in run_results.items():
            scenario_num = [k]
            intent_score = [v['intent_f1']]
            slot_score = [v['slot_f1']]
            l = scenario_num + intent_score + slot_score
            writer.writerow(l)

In [ ]:
# In case it's needed
del_all_flags(FLAGS)

In [14]:
import train
import test

results = dict()
for i in range(len(training_data_paths)):
    
    # Load data
    data = data_loader.read_datasets(w2v, training_data_paths[i], test_data_paths[i], test=True)
    
    FLAGS = set_flags(data, scenario=scenario_nums[i])
    
    # Train model
    print('TRAINING ' + scenario_nums[i])
    train.train(model_s2i.CapsNetS2I, data, FLAGS, batches_rand=False)
    
    # Test model
    print('TESTING ' + scenario_nums[i])
    intent_f, slot_f = test.test(model_s2i.CapsNetS2I, data, FLAGS)
    results[scenario_nums[i]] = dict()
    results[scenario_nums[i]]['intent_f1'] = intent_f
    results[scenario_nums[i]]['slot_f1'] = slot_f
        
    # Reset flags
    del_all_flags(FLAGS)

write_csv(results)

------------------read datasets begin-------------------
max length is 15
max length is 15
------------------read datasets end---------------------
TRAINING 0-1-attention-head
------------------start cross-validation-------------------
FOLD 1
Initializing Variables
using pre-trained word emebedding.begin...
using pre-trained word emebedding.ended...
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.071429
F score 0.071429
Slot filling
F1 score: 0.010949
Accuracy: 0.136667
------------------epoch :  0  Loss:  14.382395 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.602381
F score 0.602381
Slot filling
F1 score: 0.467933
Accuracy: 0.932857
Current F score mean 0.5351572220337066
Best F score mean 0.5351572220337066
------------------epoch :  1  Loss:  3.78336 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.840476
F score 0.840476
Slot filling
F1 

------------------epoch :  5  Loss:  0.015158908 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.961905
F score 0.961905
Slot filling
F1 score: 0.989837
Accuracy: 0.999048
Current F score mean 0.9758710801393728
Best F score mean 0.9877284690744261
------------------epoch :  6  Loss:  0.0143149225 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.973810
F score 0.973810
Slot filling
F1 score: 0.990844
Accuracy: 0.999206
Current F score mean 0.9823269389139175
Best F score mean 0.9877284690744261
------------------epoch :  7  Loss:  0.008172076 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.976190
F score 0.976190
Slot filling
F1 score: 0.992893
Accuracy: 0.999365
Current F score mean 0.9845419386028522
Best F score mean 0.9877284690744261
------------------epoch :  8  Loss:  0.007274826 ----------------------
           V

------------------epoch :  11  Loss:  0.003945297 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.992857
F score 0.992857
Slot filling
F1 score: 0.978809
Accuracy: 0.998571
Current F score mean 0.9858332132045553
Best F score mean 0.9966007176801475
------------------epoch :  12  Loss:  0.0035261055 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.992857
F score 0.992857
Slot filling
F1 score: 0.978809
Accuracy: 0.998571
Current F score mean 0.9858332132045553
Best F score mean 0.9966007176801475
------------------epoch :  13  Loss:  0.003208344 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.992857
F score 0.992857
Slot filling
F1 score: 0.978809
Accuracy: 0.998571
Current F score mean 0.9858332132045553
Best F score mean 0.9966007176801475
------------------epoch :  14  Loss:  0.002958727 ----------------------
        

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

...

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.989899
F score 0.989899
Slot filling
F1 score: 0.981763
Accuracy: 0.996032
Current F score mean 0.9858309539160603
Best F score mean 0.9871292348599459
------------------epoch :  11  Loss:  0.015740229 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.989899
F score 0.989899
Slot filling
F1 score: 0.981763
Accuracy: 0.996032
Current F score mean 0.9858309539160603
Best F score mean 0.9871292348599459
------------------epoch :  12  Loss:  0.013383767 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.989899
F score 0.989899
Slot filling
F1 score: 0.981763
Accuracy: 0.996032
Current F score mean 0.9858309539160603
Best F score mean 0.9871292348599459
------------------epoch :  13  Loss:  0.011644047 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.9

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

Intent F1: 0.277281
Slot F1: 0.454936
